In [5]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [3]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_wv.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #3365

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,963053,963053,4473367300,4/29/20,390,PPP,GREENBRIAR HOTEL CORPORATION,"101 Main Street, West",WHITE SULPHUR SPRINGS,WV,...,Unanswered,NaN,9029298.43,8/25/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276
1,963054,963054,5936297000,4/6/20,390,PPP,"THE THRASHER GROUP, INC.",600 White Oaks Blvd,BRIDGEPORT,WV,...,Non-Veteran,NaN,8042989.32,6/17/21,54.0,"Professional, Scientific, and Technical Services",WEST VIRGINIA DISTRICT OFFICE,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399
2,963055,963055,3467218209,8/4/20,390,PPP,"HALL DRILLING, LLC",1137 E. Washington Ave,ELLENBORO,WV,...,Unanswered,NaN,6935266.00,8/11/21,21.0,Mining,WEST VIRGINIA DISTRICT OFFICE,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076
3,963056,963056,2180937106,4/10/20,390,PPP,TRITON CONSTRUCTION INC,1944 WINFIELD RD,SAINT ALBANS,WV,...,Unanswered,NaN,6660844.67,6/11/21,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750
4,963057,963057,5506907200,4/27/20,390,PPP,PLEASANT VALLEY HOSPITAL INC,2520 VALLEY DR,POINT PLEASANT,WV,...,Unanswered,NaN,6597064.46,6/11/21,62.0,Health Care and Social Assistance,WEST VIRGINIA DISTRICT OFFICE,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #1

,LoanNumber,full_add,Lat,Long
2010,7369837107,"1020 McDermott Ridge Road, Buckhannon, WV",NaN,NaN


In [6]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head()

,LoanNumber,full_add,Lat,Long
0,4473367300,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276
1,5936297000,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399
2,3467218209,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076
3,2180937106,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750
4,5506907200,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/WV_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [7]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [10]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #7204

<h4>FIPS Script

In [7]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [8]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('54')] #WV
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

3364


Matching FIPS to DataFrame

In [11]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:7204]

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-12-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,4473367300,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,[540259501012016]
1,5936297000,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,[540330320002080]
2,3467218209,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,[540859623003039]
3,2180937106,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,[540790206091045]
4,5506907200,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,[540539550004006]
...,...,...,...,...,...
3360,7867607002,"903 Grand Central Ave, VIENNA, WV",39.308604,-81.553172,[541070105012030]
3361,8194978504,"218 D St, South Charleston, WV",38.365271,-81.693659,[540390129001050]
3362,9443798505,"917 Deer Ridge Road, Davis, WV",39.052267,-79.410183,[540939653002118]
3363,9755898608,"818 W Main St, Bridgeport, WV",39.282200,-80.266586,[540330311002018]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-13-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,4473367300,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,540259501012016
1,5936297000,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,540330320002080
2,3467218209,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,540859623003039
3,2180937106,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,540790206091045
4,5506907200,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,540539550004006
...,...,...,...,...,...
3360,7867607002,"903 Grand Central Ave, VIENNA, WV",39.308604,-81.553172,541070105012030
3361,8194978504,"218 D St, South Charleston, WV",38.365271,-81.693659,540390129001050
3362,9443798505,"917 Deer Ridge Road, Davis, WV",39.052267,-79.410183,540939653002118
3363,9755898608,"818 W Main St, Bridgeport, WV",39.282200,-80.266586,540330311002018


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #8

,LoanNumber,full_add,Lat,Long,FIPS_z
33,1092927109,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1076,9253917006,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1246,7284168605,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,NaN
1377,6167388303,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
1378,5082197000,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,NaN
2472,6046968304,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
3164,5550167102,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
3186,1014389001,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,4473367300,540259501012016
1,5936297000,540330320002080
2,3467218209,540859623003039
3,2180937106,540790206091045
4,5506907200,540539550004006
...,...,...
3360,7867607002,541070105012030
3361,8194978504,540390129001050
3362,9443798505,540939653002118
3363,9755898608,540330311002018


Read in base data

In [16]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_wv.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/WV_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 

#fips_check.head() #9179


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,963053,963053,4473367300,4/29/20,390,PPP,GREENBRIAR HOTEL CORPORATION,"101 Main Street, West",WHITE SULPHUR SPRINGS,WV,...,Unanswered,NaN,9029298.43,8/25/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276
1,963054,963054,5936297000,4/6/20,390,PPP,"THE THRASHER GROUP, INC.",600 White Oaks Blvd,BRIDGEPORT,WV,...,Non-Veteran,NaN,8042989.32,6/17/21,54.0,"Professional, Scientific, and Technical Services",WEST VIRGINIA DISTRICT OFFICE,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399
2,963055,963055,3467218209,8/4/20,390,PPP,"HALL DRILLING, LLC",1137 E. Washington Ave,ELLENBORO,WV,...,Unanswered,NaN,6935266.00,8/11/21,21.0,Mining,WEST VIRGINIA DISTRICT OFFICE,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076
3,963056,963056,2180937106,4/10/20,390,PPP,TRITON CONSTRUCTION INC,1944 WINFIELD RD,SAINT ALBANS,WV,...,Unanswered,NaN,6660844.67,6/11/21,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750
4,963057,963057,5506907200,4/27/20,390,PPP,PLEASANT VALLEY HOSPITAL INC,2520 VALLEY DR,POINT PLEASANT,WV,...,Unanswered,NaN,6597064.46,6/11/21,62.0,Health Care and Social Assistance,WEST VIRGINIA DISTRICT OFFICE,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,966413,966413,7867607002,4/8/20,390,PPP,T.M.T. 2 ENTERPRISES,903 Grand Central Ave,VIENNA,WV,...,Unanswered,NaN,151454.17,4/8/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"903 Grand Central Ave, VIENNA, WV",39.308604,-81.553172
3361,966414,966414,8194978504,3/9/21,390,PPS,DAMOUS PSYCHOLOGICAL SERVICES PLLC,218 D St,South Charleston,WV,...,Non-Veteran,NaN,NaN,NaN,62.0,Health Care and Social Assistance,WEST VIRGINIA DISTRICT OFFICE,"218 D St, South Charleston, WV",38.365271,-81.693659
3362,966415,966415,9443798505,3/12/21,390,PPS,STUCORP,917 Deer Ridge Road,Davis,WV,...,Non-Veteran,NaN,150731.51,9/10/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"917 Deer Ridge Road, Davis, WV",39.052267,-79.410183
3363,966416,966416,9755898608,3/26/21,390,PPS,TWIN OAKS RESTAURANT & SUPPER CLUB,818 W Main St,Bridgeport,WV,...,Non-Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"818 W Main St, Bridgeport, WV",39.282200,-80.266586


Merge with MainFrame

In [17]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,963053,963053,4473367300,4/29/20,390,PPP,GREENBRIAR HOTEL CORPORATION,"101 Main Street, West",WHITE SULPHUR SPRINGS,WV,...,NaN,9029298.43,8/25/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,540259501012016
1,963054,963054,5936297000,4/6/20,390,PPP,"THE THRASHER GROUP, INC.",600 White Oaks Blvd,BRIDGEPORT,WV,...,NaN,8042989.32,6/17/21,54.0,"Professional, Scientific, and Technical Services",WEST VIRGINIA DISTRICT OFFICE,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,540330320002080
2,963055,963055,3467218209,8/4/20,390,PPP,"HALL DRILLING, LLC",1137 E. Washington Ave,ELLENBORO,WV,...,NaN,6935266.00,8/11/21,21.0,Mining,WEST VIRGINIA DISTRICT OFFICE,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,540859623003039
3,963056,963056,2180937106,4/10/20,390,PPP,TRITON CONSTRUCTION INC,1944 WINFIELD RD,SAINT ALBANS,WV,...,NaN,6660844.67,6/11/21,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,540790206091045
4,963057,963057,5506907200,4/27/20,390,PPP,PLEASANT VALLEY HOSPITAL INC,2520 VALLEY DR,POINT PLEASANT,WV,...,NaN,6597064.46,6/11/21,62.0,Health Care and Social Assistance,WEST VIRGINIA DISTRICT OFFICE,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,540539550004006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,966413,966413,7867607002,4/8/20,390,PPP,T.M.T. 2 ENTERPRISES,903 Grand Central Ave,VIENNA,WV,...,NaN,151454.17,4/8/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"903 Grand Central Ave, VIENNA, WV",39.308604,-81.553172,541070105012030
3361,966414,966414,8194978504,3/9/21,390,PPS,DAMOUS PSYCHOLOGICAL SERVICES PLLC,218 D St,South Charleston,WV,...,NaN,NaN,NaN,62.0,Health Care and Social Assistance,WEST VIRGINIA DISTRICT OFFICE,"218 D St, South Charleston, WV",38.365271,-81.693659,540390129001050
3362,966415,966415,9443798505,3/12/21,390,PPS,STUCORP,917 Deer Ridge Road,Davis,WV,...,NaN,150731.51,9/10/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"917 Deer Ridge Road, Davis, WV",39.052267,-79.410183,540939653002118
3363,966416,966416,9755898608,3/26/21,390,PPS,TWIN OAKS RESTAURANT & SUPPER CLUB,818 W Main St,Bridgeport,WV,...,NaN,NaN,NaN,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"818 W Main St, Bridgeport, WV",39.282200,-80.266586,540330311002018


<b>Fill FIPS Columns - if FIPS file

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

In [ ]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #16 , notna #9163 / 9179

Convert to File

In [18]:
fips_WV = fips_merge
#fips_WV = fips_merge_drop

In [5]:
geo_WV_fips.to_csv("data/state_data/geo/geo_fips/WV_fips_scraped.csv") 

Review & Compare

In [6]:
geo_WV_fips = pd.read_csv("data/state_data/geo/geo_fips/WV_fips_scraped.csv") 
geo_WV_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,0,963053,963053,4473367300,4/29/20,390,PPP,...,N,9029298.43,8/25/21,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,5.402595e+14
1,1,1,1,1,963054,963054,5936297000,4/6/20,390,PPP,...,N,8042989.32,6/17/21,54.0,"Professional, Scientific, and Technical Services",WEST VIRGINIA DISTRICT OFFICE,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,5.403303e+14
2,2,2,2,2,963055,963055,3467218209,8/4/20,390,PPP,...,N,6935266.00,8/11/21,21.0,Mining,WEST VIRGINIA DISTRICT OFFICE,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,5.408596e+14
3,3,3,3,3,963056,963056,2180937106,4/10/20,390,PPP,...,N,6660844.67,6/11/21,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,5.407902e+14
4,4,4,4,4,963057,963057,5506907200,4/27/20,390,PPP,...,N,6597064.46,6/11/21,62.0,Health Care and Social Assistance,WEST VIRGINIA DISTRICT OFFICE,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,5.405396e+14


In [7]:
##### fill in missing Lat, Long, FIPS
geo_WV_fips.loc[geo_WV_fips["FIPS_z"].isnull()] #9

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
33,33,33,33,33,963086,963086,1092927109,4/9/20,390,PPP,...,N,3540016.04,9/2/21,54.0,"Professional, Scientific, and Technical Services",WEST VIRGINIA DISTRICT OFFICE,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1076,1076,1076,1076,1076,964129,964129,9253917006,4/9/20,390,PPP,...,N,435615.96,5/20/21,81.0,Other Services (except Public Administration),WEST VIRGINIA DISTRICT OFFICE,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1246,1246,1246,1246,1246,964299,964299,7284168605,3/23/21,390,PPP,...,Y,379684.11,7/9/21,71.0,"Arts, Entertainment, and Recreation",WEST VIRGINIA DISTRICT OFFICE,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,NaN
1377,1377,1377,1377,1377,964430,964430,6167388303,1/26/21,390,PPS,...,N,NaN,NaN,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
1378,1378,1378,1378,1378,964431,964431,5082197000,4/5/20,390,PPP,...,N,351728.00,8/16/21,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,NaN
2010,2010,2010,2010,2010,965063,965063,7369837107,4/14/20,390,PPP,...,N,247785.12,6/29/21,23.0,Construction,WEST VIRGINIA DISTRICT OFFICE,"1020 McDermott Ridge Road, Buckhannon, WV",NaN,NaN,NaN
2472,2472,2472,2472,2472,965525,965525,6046968304,1/26/21,390,PPP,...,N,NaN,NaN,32.0,Manufacturing,WEST VIRGINIA DISTRICT OFFICE,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
3164,3164,3164,3164,3164,966217,966217,5550167102,4/13/20,390,PPP,...,N,160128.95,7/27/21,33.0,Manufacturing,WEST VIRGINIA DISTRICT OFFICE,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
3186,3186,3186,3186,3186,966239,966239,1014389001,5/12/21,390,PPS,...,N,NaN,NaN,72.0,Accomodation and Food Services,WEST VIRGINIA DISTRICT OFFICE,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,NaN
